In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json, os, time

import pyautogui
from PIL import Image
import cv2
import numpy as np
from dotenv import load_dotenv
import openai

from prompts import *
from GUIUtils import *
from GUIToolbox import LightroomGUIToolBox
from Chat import AgentClient
from cognitive_architecture import adjustment_routing_woKI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

## First Step: Set the window, set clibration points for automated GUI operations
Set up window and calibration points for automated GUI control
This step needs to be done once and the configuration will be saved for future use
Steps:
1. Position and size the Lightroom window appropriately 
2. Follow prompts to calibrate control points for automation
3. Configuration will be saved to a file for reuse
4. Remeber to give terminal right to control your computer

In [3]:
# GUI_config_file = "./GUI_config_half.json"
GUI_config_file = "./GUI_config.json"
update_setting_image = 'UpdateSetting.png'

if os.path.exists(GUI_config_file):
    print(f"Configuration file '{GUI_config_file}' found. Loading...")
    with open(GUI_config_file, "r") as config_file:
        GUI_config = json.load(config_file)
else:
    button_image = os.path.abspath("./button.png")
    color_button_image = os.path.abspath("./color-button.png")
    HDPI = 2
    background_color = [26, 26, 26]
    pannel_and_histogram_area = set_calibration_points()
    slider_positions = record_slider_positions(button_image)
    color_slider_positions = record_color_slider_positions(color_button_image)
    GUI_config = create_toolbox_config_template(button_image, color_button_image, HDPI, background_color, pannel_and_histogram_area, slider_positions, color_slider_positions)
    with open(GUI_config_file, "w") as config_file:
        json.dump(GUI_config, config_file, indent=4)

Configuration file './GUI_config.json' found. Loading...


### Run test to control the GUI Lightroom

In [7]:
skip_legacy_setting_button(update_setting_image, GUI_config)
set_slider_positions(test_parameters_2, GUI_config)
set_slider_positions(test_parameters_3, GUI_config)
set_slider_positions(test_parameters_1, GUI_config)

set_color_slider_positions(color_test_parameters_2, GUI_config)
set_color_slider_positions(color_test_parameters_3, GUI_config)
set_color_slider_positions(color_test_parameters_1, GUI_config)

No Update Legacy Setting Button


## Test single image

In [4]:
exp_index = 60
skip_legacy_setting_button(update_setting_image, GUI_config)
set_slider_positions(test_parameters_1, GUI_config)
set_color_slider_positions(color_test_parameters_1, GUI_config)
output_dir_path = f"./test-output/P./test-output/PhotoArtAgent_V.Jan5_woKI_artistic_emphasis/test_image_withcolor_{exp_index}"
GUIToolbox = LightroomGUIToolBox(GUI_config, output_dir_path, image_name=f"test_image{exp_index}", clip_history_messages=True)
chat_client = AgentClient(api_key=api_key, toolbox_instance=GUIToolbox, debug=False)
global_style = photo_editing_global_styles["artistic_emphasis"]

No Update Legacy Setting Button
User messgae file path: /Users/jinjingu/Documents/Projects/AgentRetouching/test-output/P./test-output/PhotoArtAgent_V.Jan5_woKI_artistic_emphasis/test_image_withcolor_60/user_messages.json


If you need to view the messages. Now select the output messages.json file on the server.

In [5]:
adjustment_routing_woKI(chat_client, GUIToolbox, global_style=global_style, retry=5)

`func_return_responses`, response: 

### Image Content Description:
The image captures a roadside view at ground level, emphasizing a red curb and a shallow stream of water running alongside it. The background is slightly blurred, revealing trees, some parked cars, and hints of suburban buildings under an overcast sky. The focus is on the curb and water, while the background offers soft, natural tones with light greenery.

### Retouching Approaches:

#### Approach 1: Balanced and Subtle
- **Light:** Slightly increase overall exposure and fine-tune contrast to clarify the texture of the curb and water. Soften highlights and lift shadows for balance.
- **Color:** Adjust the temperature slightly warmer for a neutral feeling. Enhance vibrance without over-saturating.
- **Individual Colors:** Boost the red curb’s saturation slightly and improve its luminance to make it subtly pop. Add mild saturation to greens in the background for a natural balance.

#### Approach 2: Moody and Cool
- **Lig

## Run batch images

In [ ]:
processing_image_number = 114
for i in range(processing_image_number):
    exp_index = i
    skip_legacy_setting_button(update_setting_image, GUI_config)
    set_slider_positions(test_parameters_1, GUI_config)
    set_color_slider_positions(color_test_parameters_1, GUI_config)
    output_dir_path = f"./test-output/PhotoArtAgent_V.Jan5_woKI/test_image_withcolor_{exp_index}"
    GUIToolbox = LightroomGUIToolBox(GUI_config, output_dir_path, image_name=f"test_image{exp_index}", clip_history_messages=True)
    chat_client = AgentClient(api_key=api_key, toolbox_instance=GUIToolbox, debug=False)
    global_style = photo_editing_global_styles["artistic_emphasis"]

    adjustment_routing_woKI(chat_client, GUIToolbox, global_style=global_style, retry=5)

    time.sleep(2)
    pyautogui.press('right')
    time.sleep(1)

No Update Legacy Setting Button
User messgae file path: /Users/jinjingu/Documents/Projects/AgentRetouching/test-output/Ph./test-output/PhotoArtAgent_V.Jan5_woKI/test_image_withcolor_0/user_messages.json
`func_return_responses`, response: 

### Image Content Description:
The image depicts an assortment of floral bouquets arranged closely together. The central focus is a vibrant bouquet of yellow roses, surrounded by other bouquets containing different rose colors like soft pink, orange, cream, and white. The green ferns add a fresh, natural feel to the arrangement. The background is subtle, allowing the flowers to dominate the composition.

### Retouching Approaches:

**Approach 1:** Balanced and Subtle Mood  
- **Light Adjustments:** Slightly increase exposure and highlights to make the flowers appear brighter. Subtly lower shadows for depth.
- **Color Adjustments:** Boost vibrance slightly to make colors pop, keeping saturation natural.
- **Individual Color Adjustments:** Increase the